In [1]:
import cv2
import os


In [2]:
from detectron2.data.datasets import register_coco_instances
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.data.datasets import load_coco_json
from pycocotools.coco import COCO
from copy import deepcopy

img_root = "/workspace/datasets/seg_by_patient/preprocessed/pos_cropped_patch_all_r1_r2"

coco_anno_root = "/workspace/datasets/seg_by_patient/preprocessed/anno/coco_format/"
all_anno = os.path.join(coco_anno_root, "pre_all_complete_merge.json")


In [4]:
ods = load_coco_json(all_anno, img_root, "all")
nds = load_coco_json(all_anno, img_root, "all", extra_annotation_keys=["patient_id"])


Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.




Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.



In [5]:
ods[0]#["annotations"][0]

{'file_name': '/workspace/datasets/seg_by_patient/preprocessed/pos_cropped_patch_all_r1_r2/raw1_01.D2(30m)-1.JPG',
 'height': 1200,
 'width': 600,
 'image_id': 0,
 'annotations': [{'bbox': [285, 23, 92, 92],
   'category_id': 0,
   'segmentation': [[293.4530547170928,
     31.409565453390474,
     374.82228905263577,
     31.409565453390474,
     374.82228905263577,
     105.12405691794979,
     288.7766520182292,
     106.92197188590337]],
   'bbox_mode': <BoxMode.XYWH_ABS: 1>},
  {'bbox': [277, 234, 94, 94],
   'category_id': 0,
   'segmentation': [[285.0355360243056,
     237.270762619463,
     365.469514480745,
     242.66450752332372,
     358.9225691978378,
     325.3685812348301,
     283.16499344026204,
     319.97482151661103]],
   'bbox_mode': <BoxMode.XYWH_ABS: 1>},
  {'bbox': [269, 471, 96, 96],
   'category_id': 0,
   'segmentation': [[276.61801733151833,
     479.98920922140474,
     357.98728249289775,
     474.5954791319023,
     354.24616649897416,
     559.09743818264

In [6]:
nds[0]#["annotations"][0]

{'file_name': '/workspace/datasets/seg_by_patient/preprocessed/pos_cropped_patch_all_r1_r2/raw1_01.D2(30m)-1.JPG',
 'height': 1200,
 'width': 600,
 'image_id': 0,
 'annotations': [{'bbox': [285, 23, 92, 92],
   'category_id': 0,
   'patient_id': 'raw1_01',
   'segmentation': [[293.4530547170928,
     31.409565453390474,
     374.82228905263577,
     31.409565453390474,
     374.82228905263577,
     105.12405691794979,
     288.7766520182292,
     106.92197188590337]],
   'bbox_mode': <BoxMode.XYWH_ABS: 1>},
  {'bbox': [277, 234, 94, 94],
   'category_id': 0,
   'patient_id': 'raw1_01',
   'segmentation': [[285.0355360243056,
     237.270762619463,
     365.469514480745,
     242.66450752332372,
     358.9225691978378,
     325.3685812348301,
     283.16499344026204,
     319.97482151661103]],
   'bbox_mode': <BoxMode.XYWH_ABS: 1>},
  {'bbox': [269, 471, 96, 96],
   'category_id': 0,
   'patient_id': 'raw1_01',
   'segmentation': [[276.61801733151833,
     479.98920922140474,
     357.9

In [ ]:

def load_coco(json_file, img_root):
    ds_dicts = load_coco_json(json_file, img_root)

    coco = COCO(json_file)
    ids = {img_id: coco.imgs[img_id].get("patient_id") for img_id in coco.imgs}

    out = []
    for d in ds_dicts:
        d = deepcopy(d)
        img_id = d["image_id"]
        d["patient_id"] = ids[img_id]
        out.append(d)

    return out

DATASET_NAME = "all_ds"

# DatasetCatalog.remove("all_ds")
# MetadataCatalog.remove("all_ds")
if DATASET_NAME in DatasetCatalog.list():
    DatasetCatalog.remove(DATASET_NAME)

if DATASET_NAME in MetadataCatalog.list():
    MetadataCatalog.remove(DATASET_NAME)


DatasetCatalog.register(DATASET_NAME, lambda: load_coco(all_anno, img_root))

# register_coco_instances(DATASET_NAME, {}, all_anno, img_root)

all_dicts = DatasetCatalog.get("all_ds")

In [9]:
all_dicts[0]

{'file_name': '/workspace/datasets/seg_by_patient/preprocessed/pos_cropped_patch_all_r1_r2/raw1_01.D2(30m)-1.JPG',
 'height': 1200,
 'width': 600,
 'image_id': 0,
 'annotations': [{'bbox': [285, 23, 92, 92],
   'category_id': 0,
   'segmentation': [[293.4530547170928,
     31.409565453390474,
     374.82228905263577,
     31.409565453390474,
     374.82228905263577,
     105.12405691794979,
     288.7766520182292,
     106.92197188590337]],
   'bbox_mode': <BoxMode.XYWH_ABS: 1>},
  {'bbox': [277, 234, 94, 94],
   'category_id': 0,
   'segmentation': [[285.0355360243056,
     237.270762619463,
     365.469514480745,
     242.66450752332372,
     358.9225691978378,
     325.3685812348301,
     283.16499344026204,
     319.97482151661103]],
   'bbox_mode': <BoxMode.XYWH_ABS: 1>},
  {'bbox': [269, 471, 96, 96],
   'category_id': 0,
   'segmentation': [[276.61801733151833,
     479.98920922140474,
     357.98728249289775,
     474.5954791319023,
     354.24616649897416,
     559.09743818264

In [10]:
import re
import numpy as np

records = DatasetCatalog.get(DATASET_NAME)

def get_ptn_id(r):
    # if "patient_id" in r:
    return str(r["patient_id"])

    # m = re.search(r"(patient_|P)(\d+)", r["file_name"], re.IGNORECASE)
    # return m.group(2) if m else r["image_id"]  # last resort: unique per image

groups = np.array([get_ptn_id(r) for r in records])  # one per image
idx_all = np.arange(len(records))

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [13]:
from sklearn.model_selection import GroupKFold

def group_kfold_indices(groups, n_splits=5, rng_seed=42):
    rng = np.random.default_rng(rng_seed)
    uniq = np.array(sorted(set(groups)))
    rng.shuffle(uniq)
    # map each sample to the shuffled order
    order = {g:i for i,g in enumerate(uniq)}
    order_idx = np.array([order[g] for g in groups])

    gkf = GroupKFold(n_splits=n_splits)
    # NOTE: gkf ignores y, uses groups only
    for tr, te in gkf.split(np.zeros_like(order_idx), groups=groups):
        yield tr, te

def register_split(name, base_records, indices):
    subset = [base_records[i] for i in indices]
    if name in DataCatalog.list():
        DatasetCatalog.remove(name)

    DatasetCatalog.register(name, lambda s=subset: deepcopy(s))

def get_patient_ids(records):
    ptn_ids = {}

    tmp = []

    idx = 0

    for i in records:
        ptn_id = i.get("patient_id")
        if ptn_id not in tmp:
            ptn_ids[idx] = ptn_id
            idx += 1
            tmp.append(ptn_id)

        
    return ptn_ids

In [19]:
outer_split = group_kfold_indices(groups, n_splits=5)

print(len(groups))

for o_fold, (outer_tr_idx, outer_te_idx) in enumerate(outer_split):
    # print(outer_tr_idx, outer_te_idx)

    inner_split = group_kfold_indices(groups[outer_tr_idx], n_splits=3)

    print(len(groups[outer_tr_idx]))

    for i_fold, (inner_tr_rel, inner_va_rel) in enumerate(inner_split):
        print(f"Test: {outer_tr_idx}")
        print(f"Train: {inner_tr_rel}")
        print(f"valid: {inner_va_rel}")

print("="*50)




274
219
Test: [  0   1   2   3   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  40  41
  42  43  44  45  46  47  52  53  54  55  56  57  58  59  60  61  62  63
  64  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83  84
  89  90  91  92  93  94  95  96  97  98  99 100 104 105 106 107 108 109
 111 112 113 118 119 120 121 122 123 124 125 126 127 129 130 131 132 135
 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 155
 156 157 158 159 160 165 166 167 168 169 170 171 172 173 174 178 179 180
 181 182 183 184 185 186 187 188 189 190 191 195 196 197 198 199 200 201
 202 203 204 205 206 207 208 209 210 211 212 213 215 216 220 221 222 223
 224 225 226 227 228 229 230 231 236 242 243 244 245 246 247 248 249 250
 251 252 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270
 271 272 273]
Train: [  6   7   8  12  13  14  15  20  21  30  31  32  33  34  35  36  37  39
  40  41  42  43

In [20]:
groups

array(['raw1_01', 'raw1_01', 'raw1_01', 'raw1_01', 'raw1_03', 'raw1_04',
       'raw1_04', 'raw1_05', 'raw1_05', 'raw1_05', 'raw1_06', 'raw1_06',
       'raw1_06', 'raw1_07', 'raw1_07', 'raw1_07', 'raw1_07', 'raw1_08',
       'raw1_08', 'raw1_08', 'raw1_08', 'raw1_09', 'raw1_10', 'raw1_11',
       'raw1_11', 'raw1_11', 'raw1_11', 'raw1_12', 'raw1_12', 'raw1_12',
       'raw1_12', 'raw1_13', 'raw1_13', 'raw1_13', 'raw1_14', 'raw1_15',
       'raw1_15', 'raw1_16', 'raw1_16', 'raw1_16', 'raw1_17', 'raw1_17',
       'raw1_17', 'raw1_17', 'raw1_18', 'raw1_19', 'raw1_19', 'raw1_20',
       'raw1_21', 'raw1_21', 'raw1_21', 'raw1_21', 'raw1_22', 'raw1_22',
       'raw1_22', 'raw1_23', 'raw1_23', 'raw1_23', 'raw1_23', 'raw1_24',
       'raw1_24', 'raw1_24', 'raw1_24', 'raw1_25', 'raw1_25', 'raw1_26',
       'raw1_26', 'raw1_26', 'raw1_27', 'raw1_27', 'raw1_27', 'raw1_27',
       'raw1_28', 'raw1_29', 'raw1_29', 'raw1_29', 'raw1_29', 'raw1_30',
       'raw1_30', 'raw1_30', 'raw1_30', 'raw1_31', 

In [8]:
ol = group_kfold_indices(groups, n_splits=5, rng_seed=123)

In [9]:
for i in ol:
    print(i)

(array([  0,   1,   2,   3,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  40,  41,  42,  43,  44,
        45,  46,  47,  52,  53,  54,  55,  56,  57,  58,  59,  60,  61,
        62,  63,  64,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  89,  90,  91,  92,  93,  94,
        95,  96,  97,  98,  99, 100, 104, 105, 106, 107, 108, 109, 111,
       112, 113, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 129,
       130, 131, 132, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144,
       145, 146, 147, 148, 149, 150, 151, 152, 155, 156, 157, 158, 159,
       160, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 178, 179,
       180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 195,
       196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208,
       209, 210, 211, 212, 213, 215, 216, 220, 221, 222, 223, 2

In [10]:
len(records)

274

In [4]:
from detectron2.data.datasets import register_coco_instances
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.data.datasets import load_coco_json
from pycocotools.coco import COCO
from copy import deepcopy
from pathlib import Path

img_root = "/workspace/datasets/seg_by_patient/preprocessed/pos_cropped_patch_all_r1_r2"

coco_anno_root = Path("/workspace/datasets/seg_by_patient/preprocessed/anno/coco_format/")
all_anno = coco_anno_root / "pre_all_complete_merge.json"

anno = COCO(all_anno)

anno

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [7]:
ids = {img_id: anno.imgs[img_id].get("patient_id") for img_id in anno.imgs}

ids

{0: 'raw1_01',
 1: 'raw1_01',
 2: 'raw1_01',
 3: 'raw1_01',
 4: 'raw1_03',
 5: 'raw1_04',
 6: 'raw1_04',
 7: 'raw1_05',
 8: 'raw1_05',
 9: 'raw1_05',
 10: 'raw1_06',
 11: 'raw1_06',
 12: 'raw1_06',
 13: 'raw1_07',
 14: 'raw1_07',
 15: 'raw1_07',
 16: 'raw1_07',
 17: 'raw1_08',
 18: 'raw1_08',
 19: 'raw1_08',
 20: 'raw1_08',
 21: 'raw1_09',
 22: 'raw1_10',
 23: 'raw1_11',
 24: 'raw1_11',
 25: 'raw1_11',
 26: 'raw1_11',
 27: 'raw1_12',
 28: 'raw1_12',
 29: 'raw1_12',
 30: 'raw1_12',
 31: 'raw1_13',
 32: 'raw1_13',
 33: 'raw1_13',
 34: 'raw1_14',
 35: 'raw1_15',
 36: 'raw1_15',
 37: 'raw1_16',
 38: 'raw1_16',
 39: 'raw1_16',
 40: 'raw1_17',
 41: 'raw1_17',
 42: 'raw1_17',
 43: 'raw1_17',
 44: 'raw1_18',
 45: 'raw1_19',
 46: 'raw1_19',
 47: 'raw1_20',
 48: 'raw1_21',
 49: 'raw1_21',
 50: 'raw1_21',
 51: 'raw1_21',
 52: 'raw1_22',
 53: 'raw1_22',
 54: 'raw1_22',
 55: 'raw1_23',
 56: 'raw1_23',
 57: 'raw1_23',
 58: 'raw1_23',
 59: 'raw1_24',
 60: 'raw1_24',
 61: 'raw1_24',
 62: 'raw1_24',
 6

In [11]:
records

[{'file_name': '/workspace/datasets/seg_by_patient/preprocessed/pos_cropped_patch_all_r1_r2/raw1_01.D2(30m)-1.JPG',
  'height': 1200,
  'width': 600,
  'image_id': 0,
  'annotations': [{'bbox': [285, 23, 92, 92],
    'category_id': 0,
    'segmentation': [[293.4530547170928,
      31.409565453390474,
      374.82228905263577,
      31.409565453390474,
      374.82228905263577,
      105.12405691794979,
      288.7766520182292,
      106.92197188590337]],
    'bbox_mode': <BoxMode.XYWH_ABS: 1>},
   {'bbox': [277, 234, 94, 94],
    'category_id': 0,
    'segmentation': [[285.0355360243056,
      237.270762619463,
      365.469514480745,
      242.66450752332372,
      358.9225691978378,
      325.3685812348301,
      283.16499344026204,
      319.97482151661103]],
    'bbox_mode': <BoxMode.XYWH_ABS: 1>},
   {'bbox': [269, 471, 96, 96],
    'category_id': 0,
    'segmentation': [[276.61801733151833,
      479.98920922140474,
      357.98728249289775,
      474.5954791319023,
      354.246

In [14]:
print(records[0]['patient_id'])
print(records[1]['patient_id'])


raw1_01
raw1_01
